In [5]:
import numpy as np
import pandas as pd
import pickle
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [42]:
#Load Bitcoin Price History
with open('../crypto_currency_and_market_value_history/data/bitcoin_4_year_hourly_history_df.pickle', 'rb') as file:
     bitcoin_2_5_year_price_hist_df = pickle.load(file)


In [43]:
bitcoin_2_5_year_price_hist_df.head(5)

,timestamp,open,high,low,close,volume (btc),volume (currency),weighted price
0,2015-05-31 00:00:00,232.11,232.11,231.92,231.92,223.37,51822.18,232.00
1,2015-05-31 01:00:00,231.88,232.32,231.87,232.32,217.49,50452.68,231.98
2,2015-05-31 02:00:00,231.97,232.51,231.88,232.51,46.45,10782.23,232.14
3,2015-05-31 03:00:00,232.49,232.49,231.93,232.49,8.85,2057.04,232.38
4,2015-05-31 04:00:00,232.49,232.49,231.90,232.22,14.78,3429.00,232.05


In [44]:
bitcoin_2_5_year_price_hist_df = bitcoin_2_5_year_price_hist_df[bitcoin_2_5_year_price_hist_df.timestamp > '2017-06-01 00:00:00']

In [45]:
#bitcoin_2_5_year_price_hist_df = bitcoin_2_5_year_price_hist_df.iloc[::-1]

In [46]:
features = bitcoin_2_5_year_price_hist_df[['open','high','low','close']].values

In [47]:
price_variation = (1- (features[:,0]/features[:,3]))*100
highs = (features[:,1]/np.maximum(features[:,0],features[:,3]) -1)*100
lows = (features[:,2]/np.minimum(features[:,0],features[:,3]) -1)*100

In [48]:
X = np.array([price_variation , highs, lows]).transpose()
X = np.reshape(X,(X.shape[0],X.shape[1],1))
print(X[0])

[[ 1.88933014]
 [ 0.06703664]
 [-0.4748334 ]]


In [49]:
Y = np.array((np.sign((features[2:,3]/features[:-2,3]-1))+1)/2)
print(Y[:5])

[1. 1. 1. 0. 1.]


In [50]:
X.shape

(17735, 3, 1)

In [51]:
Y.shape

(17733,)

In [53]:
X_train = X[:12414]
X_test = X[12414:17733]
Y_train = Y[:12414]
Y_test = Y[12414:17733]

In [54]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(12414, 3, 1)
(5319, 3, 1)
(12414,)
(5319,)


In [55]:
model = Sequential()
model.add(LSTM(100,
               input_shape = (None,1),
               return_sequences = True
              ))
model.add(Dropout(0.1))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(50))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="mse", optimizer="rmsprop")

In [56]:
model.fit(X_train,Y_train, batch_size=512,
          epochs=500,
          validation_split=0.05)

Train on 11793 samples, validate on 621 samples
Epoch 1/500
11793/11793 [==============================] - 3s 293us/step - loss: 0.2485 - val_loss: 0.2513
Epoch 2/500
11793/11793 [==============================] - 1s 99us/step - loss: 0.2482 - val_loss: 0.2501
Epoch 3/500
11793/11793 [==============================] - 1s 98us/step - loss: 0.2481 - val_loss: 0.2502
Epoch 4/500
11793/11793 [==============================] - 1s 100us/step - loss: 0.2481 - val_loss: 0.2503
Epoch 5/500
11793/11793 [==============================] - 1s 100us/step - loss: 0.2481 - val_loss: 0.2497
Epoch 6/500
11793/11793 [==============================] - 1s 106us/step - loss: 0.2480 - val_loss: 0.2496
Epoch 7/500
11793/11793 [==============================] - 1s 98us/step - loss: 0.2480 - val_loss: 0.2503
Epoch 8/500
11793/11793 [==============================] - 1s 100us/step - loss: 0.2481 - val_loss: 0.2503
Epoch 9/500
11793/11793 [==============================] - 1s 96us/step - loss: 0.2481 - val_loss: 

11793/11793 [==============================] - 1s 108us/step - loss: 0.2471 - val_loss: 0.2465
Epoch 77/500
11793/11793 [==============================] - 1s 118us/step - loss: 0.2471 - val_loss: 0.2474
Epoch 78/500
11793/11793 [==============================] - 1s 109us/step - loss: 0.2471 - val_loss: 0.2485
Epoch 79/500
11793/11793 [==============================] - 1s 113us/step - loss: 0.2471 - val_loss: 0.2503
Epoch 80/500
11793/11793 [==============================] - 1s 103us/step - loss: 0.2471 - val_loss: 0.2474
Epoch 81/500
11793/11793 [==============================] - 1s 102us/step - loss: 0.2470 - val_loss: 0.2487
Epoch 82/500
11793/11793 [==============================] - 1s 116us/step - loss: 0.2470 - val_loss: 0.2485
Epoch 83/500
11793/11793 [==============================] - 2s 166us/step - loss: 0.2470 - val_loss: 0.2464
Epoch 84/500
11793/11793 [==============================] - 2s 142us/step - loss: 0.2470 - val_loss: 0.2461
Epoch 85/500
11793/11793 [===============

11793/11793 [==============================] - 2s 130us/step - loss: 0.2465 - val_loss: 0.2466
Epoch 152/500
11793/11793 [==============================] - 2s 143us/step - loss: 0.2465 - val_loss: 0.2466s - lo
Epoch 153/500
11793/11793 [==============================] - 2s 141us/step - loss: 0.2465 - val_loss: 0.2475
Epoch 154/500
11793/11793 [==============================] - 1s 120us/step - loss: 0.2465 - val_loss: 0.2454
Epoch 155/500
11793/11793 [==============================] - 1s 125us/step - loss: 0.2465 - val_loss: 0.2479
Epoch 156/500
11793/11793 [==============================] - ETA: 0s - loss: 0.246 - 2s 158us/step - loss: 0.2466 - val_loss: 0.2481
Epoch 157/500
11793/11793 [==============================] - 2s 185us/step - loss: 0.2466 - val_loss: 0.2470
Epoch 158/500
11793/11793 [==============================] - 2s 127us/step - loss: 0.2465 - val_loss: 0.2480
Epoch 159/500
11793/11793 [==============================] - 1s 117us/step - loss: 0.2466 - val_loss: 0.2474
Epo

11793/11793 [==============================] - 1s 98us/step - loss: 0.2464 - val_loss: 0.2465
Epoch 227/500
11793/11793 [==============================] - 1s 99us/step - loss: 0.2463 - val_loss: 0.2475
Epoch 228/500
11793/11793 [==============================] - 1s 98us/step - loss: 0.2462 - val_loss: 0.2470
Epoch 229/500
11793/11793 [==============================] - 1s 98us/step - loss: 0.2462 - val_loss: 0.2469
Epoch 230/500
11793/11793 [==============================] - 1s 98us/step - loss: 0.2461 - val_loss: 0.2467
Epoch 231/500
11793/11793 [==============================] - 1s 98us/step - loss: 0.2462 - val_loss: 0.2456
Epoch 232/500
11793/11793 [==============================] - 1s 102us/step - loss: 0.2461 - val_loss: 0.2465
Epoch 233/500
11793/11793 [==============================] - 1s 105us/step - loss: 0.2462 - val_loss: 0.2470
Epoch 234/500
11793/11793 [==============================] - 1s 111us/step - loss: 0.2462 - val_loss: 0.2471
Epoch 235/500
11793/11793 [============

11793/11793 [==============================] - 1s 104us/step - loss: 0.2456 - val_loss: 0.2457
Epoch 302/500
11793/11793 [==============================] - 1s 108us/step - loss: 0.2457 - val_loss: 0.2466
Epoch 303/500
11793/11793 [==============================] - 1s 109us/step - loss: 0.2458 - val_loss: 0.2469
Epoch 304/500
11793/11793 [==============================] - 1s 108us/step - loss: 0.2457 - val_loss: 0.2456
Epoch 305/500
11793/11793 [==============================] - 1s 107us/step - loss: 0.2458 - val_loss: 0.2451
Epoch 306/500
11793/11793 [==============================] - 1s 124us/step - loss: 0.2456 - val_loss: 0.2460
Epoch 307/500
11793/11793 [==============================] - 1s 110us/step - loss: 0.2454 - val_loss: 0.2460
Epoch 308/500
11793/11793 [==============================] - 1s 110us/step - loss: 0.2458 - val_loss: 0.2463
Epoch 309/500
11793/11793 [==============================] - 1s 109us/step - loss: 0.2457 - val_loss: 0.2475
Epoch 310/500
11793/11793 [======

11793/11793 [==============================] - 1s 108us/step - loss: 0.2451 - val_loss: 0.2465
Epoch 377/500
11793/11793 [==============================] - 1s 111us/step - loss: 0.2449 - val_loss: 0.2458
Epoch 378/500
11793/11793 [==============================] - 1s 112us/step - loss: 0.2448 - val_loss: 0.2463
Epoch 379/500
11793/11793 [==============================] - 1s 109us/step - loss: 0.2450 - val_loss: 0.2465
Epoch 380/500
11793/11793 [==============================] - 1s 107us/step - loss: 0.2449 - val_loss: 0.2463
Epoch 381/500
11793/11793 [==============================] - 1s 110us/step - loss: 0.2448 - val_loss: 0.2453
Epoch 382/500
11793/11793 [==============================] - 1s 107us/step - loss: 0.2450 - val_loss: 0.2463
Epoch 383/500
11793/11793 [==============================] - 1s 111us/step - loss: 0.2448 - val_loss: 0.2462
Epoch 384/500
11793/11793 [==============================] - 1s 107us/step - loss: 0.2445 - val_loss: 0.2464
Epoch 385/500
11793/11793 [======

11793/11793 [==============================] - 2s 135us/step - loss: 0.2437 - val_loss: 0.2451
Epoch 452/500
11793/11793 [==============================] - 1s 127us/step - loss: 0.2439 - val_loss: 0.2454
Epoch 453/500
11793/11793 [==============================] - 2s 129us/step - loss: 0.2437 - val_loss: 0.2469
Epoch 454/500
11793/11793 [==============================] - 1s 117us/step - loss: 0.2438 - val_loss: 0.2470
Epoch 455/500
11793/11793 [==============================] - 1s 127us/step - loss: 0.2438 - val_loss: 0.2463
Epoch 456/500
11793/11793 [==============================] - 2s 141us/step - loss: 0.2435 - val_loss: 0.2465
Epoch 457/500
11793/11793 [==============================] - 1s 125us/step - loss: 0.2435 - val_loss: 0.2457
Epoch 458/500
11793/11793 [==============================] - 1s 118us/step - loss: 0.2435 - val_loss: 0.2454
Epoch 459/500
11793/11793 [==============================] - 2s 139us/step - loss: 0.2433 - val_loss: 0.2465
Epoch 460/500
11793/11793 [======

In [57]:
model.evaluate(X_test,Y_test)

5319/5319 [==============================] - 1s 206us/step


0.24847058541715583